In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/Tritangent_plane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
var('xx')
Q.<i> = NumberField(xx^2+1)

In [3]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)

In [4]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [5]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [6]:
line = Line([y, z])

In [7]:
general_cubic = Cubic(cubic_new, line, sing_cubics)

In [8]:
SE1 = general_cubic.subs({b:-(c*c+e*f)/c})

In [9]:
%time adm_SE1 = SE1.find_admissible_projectivities()

CPU times: user 7.29 s, sys: 145 ms, total: 7.43 s
Wall time: 16.4 s


In [10]:
len(adm_SE1)

576

In [11]:
%time simm_SE1 = SE1.find_simmetries(adm_SE1)

CPU times: user 2.85 s, sys: 149 ms, total: 3 s
Wall time: 21.7 s


In [12]:
len(simm_SE1)

2

In [13]:
adm_perm = SE1.find_admissible_permutations()
keys = list(SE1.cl_lines.keys())

In [14]:
len(adm_perm)

1152

In [15]:
M2 = [mat for mat in simm_SE1 if mat != matrix.identity(4)][0]
M2_perm = Permutation(apply_proj_to_lines(M2, list(SE1.cl_lines.values()))).to_permutation_group_element()
print(M2_perm)

(2,19)(3,22)(4,7)(5,25)(6,26)(8,13)(9,14)(11,16)(12,17)(18,27)(20,24)(21,23)


In [16]:
perms_lines = []
for perm in adm_perm:
    perms_lines.append(Permutation([perm.index(label)+1 for label in keys]).to_cycles())

In [17]:
Gr = PermutationGroup(perms_lines)

In [18]:
Gr.order()

1152

In [19]:
M2_perm.order()

2

In [20]:
el_order_4 = []
for el in Gr.list():
    iden = Gr.identity()
    if el^2 == M2_perm and el^3 != iden and el^4 == iden:
        el_order_4.append(el)

In [21]:
len(el_order_4)

12

In [22]:
labels_el_order_4 = [from_perm_to_labels(el) for el in el_order_4]
for perm in labels_el_order_4:
    print([perm[keys.index(label)] for label in ['E1', 'G4', 'E2', 'G3', 'E3', 'E5']])

['E1', 'G4', 'E5', 'F15', 'E6', 'F24']
['E1', 'G4', 'F45', 'G5', 'F46', 'E2']
['E1', 'G4', 'E6', 'F16', 'E5', 'F34']
['E1', 'G4', 'F46', 'G6', 'F45', 'E3']
['E1', 'G4', 'E3', 'G6', 'F24', 'F46']
['E1', 'G4', 'F34', 'F16', 'E2', 'E6']
['E1', 'G4', 'E5', 'G2', 'F46', 'F24']
['E1', 'G4', 'F45', 'F12', 'E6', 'E2']
['E1', 'G4', 'E3', 'G5', 'F24', 'E6']
['E1', 'G4', 'F34', 'F15', 'E2', 'F46']
['E1', 'G4', 'E6', 'G2', 'F45', 'E3']
['E1', 'G4', 'F46', 'F12', 'E5', 'F34']


In [23]:
projs = SE1.find_admissible_projectivities(labels_el_order_4)

In [24]:
len(projs)

12

In [25]:
def find_conditions_for_subfamilies(cubic, projectivities, simmetries):
    mon = ((x+y+z+t)^3).monomials()
    conditions = []
    sing_cubics_factored = cubic.sing_cubic
    for M in [proj for proj in projectivities if proj not in simmetries]:
        print(projectivities.index(M))
        sost = change_coord(M)
        new_cubic = remove_sing_factors(cubic.eqn.subs(sost), sing_cubics_factored)    
        minor = list(set(matrix([[new_cubic.coefficient(mn) for mn in mon], [cubic.eqn.coefficient(mn) for mn in mon]]).minors(2)))
        minor = [remove_sing_factors(el, sing_cubics_factored) for el in minor if el !=0]
        ideale = cubic.P.ideal(minor)
        new_ideals = get_valid_ideals(ideale, cubic, sing_cubics_factored)
        for ideal in new_ideals:            
            prim_deco2 = (ideale + ideal).radical().primary_decomposition()
            for ideal2 in prim_deco2:
                if is_ideal_valid(ideal2, cubic, sing_cubics_factored):
                    conditions.append(ideal2.gens())                        
    return list(set(conditions))


def get_valid_ideals(ideale, cubic, sing_cubics_factored):
    valid_ideals = []
    vrs = [c,d,e,f]
    for vr in vrs:
        valid_ideals_var = []
        del_ideale = ideale.elimination_ideal([vr])
        prim_deco_del = del_ideale.radical().primary_decomposition()
        for ideal in prim_deco_del:
            if is_ideal_valid(ideal, cubic, sing_cubics_factored):
                valid_ideals_var.append(ideal)
        if valid_ideals_var == []:
            return []
        else:
            valid_ideals.append(valid_ideals_var)
    return [id1+id2+id3+id4 for id1 in valid_ideals[0] for id2 in valid_ideals[1] for id3 in valid_ideals[2] for id4 in valid_ideals[3]]           

In [26]:
%time conds = find_conditions_for_subfamilies(SE1, projs, simm_SE1)

0
1
2
3
4
5
6
7
8
9
10
11
CPU times: user 4min 26s, sys: 3.59 s, total: 4min 30s
Wall time: 4min 32s


In [27]:
conds_linear = [cond for cond in conds if [el.degree() for el in cond] == [1,1,1]]
len(conds_linear)

28

In [28]:
non_linear_conds = [cond for cond in conds if cond not in conds_linear]
len(non_linear_conds)

60

In [29]:
non_linear_conds

[[2*c + (-i + 1)*d + (-3*i + 3)*e + (i + 1)*f, 193*e^2 + (-3*i + 50)*d*f + (-11*i + 119)*e*f + (65*i - 54)*f^2, 193*d*e + (41*i - 233)*d*f + (279*i - 597)*e*f + (-245*i + 159)*f^2, 193*d^2 + (83*i + 804)*d*f + (-725*i + 1983)*e*f + (775*i - 629)*f^2],
 [17*d + (4*i - 1)*e + (-2*i + 9)*f, 17*c + (-4*i + 1)*e + (2*i + 8)*f, e^2 + (12*i + 2)*e*f + (-4*i - 5)*f^2],
 [250*e^2 + (26*i + 268)*c*f + (27*i - 39)*d*f + (83*i + 19)*e*f + (-99*i - 107)*f^2, 50*d*e + (22*i + 16)*c*f + (19*i + 7)*d*f + (-29*i + 43)*e*f + (57*i + 11)*f^2, 10*c*e + (18*i - 6)*c*f + (i + 3)*d*f + (9*i + 7)*e*f + (3*i - 1)*f^2, 50*d^2 + (-78*i - 76)*c*f + (39*i + 13)*d*f + (39*i - 117)*e*f + (-39*i + 47)*f^2, 10*c*d + (-2*i - 4)*c*f + (i - 3)*d*f + (i - 3)*e*f + (-i - 7)*f^2, 2*c^2 + (-2*i - 4)*c*f + (i - 1)*d*f + (i - 3)*e*f + (-i - 1)*f^2],
 [327*d*e + (-6*i - 129)*e^2 + (-14*i + 26)*c*f + (48*i - 167)*d*f + (56*i - 104)*e*f + (-32*i + 75)*f^2, 654*c*e + (504*i + 372)*e^2 + (-23*i - 331)*c*f + (-108*i + 76)*d*f + (-56

In [30]:
sost = solve_linear_system(conds_linear[0], [d,e,f], [c])
sost

[(-169*i + 1352)*c, (169*i + 845)*c, 2197*c, (-338*i + 507)*c]

In [31]:
var = [d,e,f,c]

In [32]:
nc = SE1.subs({var[i]:sost[i] for i in range(4)})
nc

(742586*i + 3712930)*x^2*y + (-742586*i - 3712930)*x*y^2 + (913952*i - 3598686)*x^2*z + (-1485172*i + 2227758)*x*y*z + (1256684*i + 1085318)*x*z^2 + (-971074*i - 399854)*x*y*t + (399854*i - 971074)*y^2*t + (571220*i + 1370928)*y*z*t + (-685464*i + 285610)*y*t^2

In [33]:
nc.eckardt_points

[(0, 0, 0, 1)]

In [34]:
%time nc_proj = nc.find_admissible_projectivities()

CPU times: user 971 ms, sys: 89.4 ms, total: 1.06 s
Wall time: 9.99 s


In [35]:
len(nc_proj)

576

In [37]:
%time nc_simm = nc.find_simmetries(nc_proj)

CPU times: user 188 ms, sys: 110 ms, total: 297 ms
Wall time: 1.23 s


In [38]:
len(nc_simm)

4

In [39]:
for i in range(len(nc_simm)-1):
    mat1 = nc_simm[i]
    for j in range(i+1, len(nc_simm)):
        mat2 = nc_simm[j]
        are_matrices_equal(mat1, mat2)

In [40]:
Gnc = Group(nc_simm)

In [41]:
Gnc.get_divisors_of_order()

[1, 2, 4]

In [42]:
Gnc.get_order_of_elements()

[1, 4, 4, 2]

In [43]:
Gnc.is_group()

True

In [ ]:
%%time
for cond in conds_linear:
    sost = solve_linear_system(cond, [d,e,f], [c])
    nc = SE1.subs({var[i]:sost[i] for i in range(4)})
    print(len(nc.find_simmetries()))

In [45]:
with open('conditions.pickle', 'rb') as ff:
    conditions = pickle.load(ff)